## Using wrappers for Scikit learn API

This tutorial is about using gensim models as a part of your scikit learn workflow with the help of wrappers found at ```gensim.sklearn_integration.SklearnWrapperGensimLdaModel```

The wrapper available (as of now) are :
* LdaModel (```gensim.sklearn_integration.SklearnWrapperGensimLdaModel.SklearnWrapperLdaModel```),which implements gensim's ```LdaModel``` in a scikit-learn interface

### LdaModel

To use LdaModel begin with importing LdaModel wrapper

In [1]:
from gensim.sklearn_integration.SklearnWrapperGensimLdaModel import SklearnWrapperLdaModel

Next we will create a dummy set of texts and convert it into a corpus

In [2]:
from gensim.corpora import Dictionary
texts = [['complier', 'system', 'computer'],
 ['eulerian', 'node', 'cycle', 'graph', 'tree', 'path'],
 ['graph', 'flow', 'network', 'graph'],
 ['loading', 'computer', 'system'],
 ['user', 'server', 'system'],
 ['tree','hamiltonian'],
 ['graph', 'trees'],
 ['computer', 'kernel', 'malfunction','computer'],
 ['server','system','computer']]
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

Then to run the LdaModel on it

In [3]:
model=SklearnWrapperLdaModel(num_topics=2,id2word=dictionary,iterations=20, random_state=1)
model.fit(corpus)
model.print_topics(2)

[(0,
  u'0.164*"computer" + 0.117*"system" + 0.105*"graph" + 0.061*"server" + 0.057*"tree" + 0.046*"malfunction" + 0.045*"kernel" + 0.045*"complier" + 0.043*"loading" + 0.039*"hamiltonian"'),
 (1,
  u'0.102*"graph" + 0.083*"system" + 0.072*"tree" + 0.064*"server" + 0.059*"user" + 0.059*"computer" + 0.057*"trees" + 0.056*"eulerian" + 0.055*"node" + 0.052*"flow"')]

### Integration with Sklearn

To provide a better example of how it can be used with Sklearn, Let's use CountVectorizer method of sklearn. For this example we will use [20 Newsgroups data set](http://qwone.com/~jason/20Newsgroups/). We will only use the categories rec.sport.baseball and sci.crypt and use it to generate topics.

In [4]:
import numpy as np
from gensim import matutils
from gensim.models.ldamodel import LdaModel
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from gensim.sklearn_integration.SklearnWrapperGensimLdaModel import SklearnWrapperLdaModel

In [5]:
rand = np.random.mtrand.RandomState(1) # set seed for getting same result
cats = ['rec.sport.baseball', 'sci.crypt']
data = fetch_20newsgroups(subset='train',
                        categories=cats,
                        shuffle=True)

Next, we use countvectorizer to convert the collection of text documents to a matrix of token counts.

In [6]:
vec = CountVectorizer(min_df=10, stop_words='english')

X = vec.fit_transform(data.data)
vocab = vec.get_feature_names() #vocab to be converted to id2word 

id2word=dict([(i, s) for i, s in enumerate(vocab)])

Next, we just need to fit X and id2word to our Lda wrapper.

In [7]:
obj=SklearnWrapperLdaModel(id2word=id2word,num_topics=5,passes=20)
lda=obj.fit_predict(X)
lda.print_topics()

[(0,
  u'0.027*"accurate" + 0.022*"corporate" + 0.012*"consideration" + 0.012*"chance" + 0.011*"decipher" + 0.011*"example" + 0.010*"basically" + 0.010*"cases" + 0.010*"follow" + 0.009*"dawson"'),
 (1,
  u'0.018*"face" + 0.012*"fierkelab" + 0.009*"cryptography" + 0.008*"abuse" + 0.007*"150" + 0.007*"finish" + 0.006*"communication" + 0.006*"fast" + 0.006*"constitutional" + 0.006*"database"'),
 (2,
  u'0.074*"abroad" + 0.066*"asking" + 0.055*"cryptography" + 0.030*"arithmetic" + 0.028*"argue" + 0.028*"ciphertext" + 0.025*"456" + 0.023*"courtesy" + 0.019*"facts" + 0.015*"beastmaster"'),
 (3,
  u'0.014*"cryptography" + 0.014*"clark" + 0.012*"authentication" + 0.008*"corporate" + 0.008*"1993apr5" + 0.007*"brett" + 0.006*"acceptable" + 0.006*"chance" + 0.006*"considering" + 0.006*"accurate"'),
 (4,
  u'0.051*"certain" + 0.043*"book" + 0.041*"69" + 0.040*"demand" + 0.040*"87" + 0.039*"cracking" + 0.039*"abroad" + 0.037*"farm" + 0.019*"asking" + 0.015*"cryptosystems"')]

#### Comparison to Sklearn's Logistic Regression

Now lets try Sklearn's logistic classifier to classify the given categories into two types.Ideally we should get postive weights when cryptography is talked about and negative when baseball is talked about.

In [8]:
from sklearn import linear_model

In [9]:
def print_features(clf, vocab, n=10):
    ''' Better printing for sorted list '''
    coef = clf.coef_[0]
    print 'Positive features: %s' % (' '.join(['%s:%.2f' % (vocab[j], coef[j]) for j in np.argsort(coef)[::-1][:n] if coef[j] > 0]))
    print 'Negative features: %s' % (' '.join(['%s:%.2f' % (vocab[j], coef[j]) for j in np.argsort(coef)[:n] if coef[j] < 0]))

In [10]:
clf=linear_model.LogisticRegression(penalty='l1', C=0.1) #l1 penalty used
clf.fit(X,data.target)
print_features(clf,vocab)

Positive features: clipper:1.49 code:1.23 key:1.04 encryption:0.95 government:0.37 chip:0.37 nsa:0.37 uk:0.36 org:0.23 cryptography:0.23
Negative features: baseball:-1.33 game:-0.72 year:-0.61 team:-0.38 edu:-0.27 games:-0.27 players:-0.23 ball:-0.17 season:-0.14 phillies:-0.11
